In [1]:
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from sbi_scraping import get_own_stock_df
from minkabu_scraping import append_dividend
from tse_stock_list_downloader import append_market_and_industries

import config

import pandas as pd
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
# 通常のドライバ
driver = webdriver.Chrome(
    service=ChromeService(ChromeDriverManager().install())
)

In [3]:
try:
    # 証券会社のwebサイトから保有株情報を抽出する
    df_own_stock = get_own_stock_df(driver)
except:
    print("Failed get_own_stock_df")
finally:
    driver.quit()

SBI scraping started...
Minkabu scraping finished


In [6]:
try:
    # 配当金額を抽出する（処理に時間がかかる）
    df_own_stock = append_dividend(df_own_stock)
    # このあとでエラーになった場合途中から再開できるよう、CSVに吐き出しておく
    df_own_stock.to_csv("own_stock.csv")
except:
    print("Failed get_own_stock_df")

Minkabu scraping started...
1434 start1518 start

1775 start
1820 start
1835 start
1925 start
1928 start
1951 start
2124 start
2127 start
2169 start
2317 start
1434 finish
2353 start
1835 finish
2395 start
2124 finish
2914 start
1820 finish
2927 start
1951 finish
3003 start
2317 finish
3101 start
1518 skipped
3179 start
1775 skipped
3186 start
1928 skipped
3248 start
2169 skipped
3409 start
1925 skipped
3632 start
2914 skipped
3817 start
2127 skipped
3847 start
2353 skipped
4008 start
2395 skipped
4042 start
3101 skipped
4044 start
3179 skipped
4064 start
2927 skipped
4182 start
3003 skipped
4361 start
3186 skipped
4452 start
3409 skipped
4502 start
3632 skipped
4641 start
3248 skipped
5101 start
4044 skipped
5184 start
5201 start
5406 start
3817 skipped
5803 start
4452 skipped
6073 start
5184 skipped
6087 start
6073 finish
6113 start
3847 skipped
6167 start
4008 skipped
6291 start
4182 skipped
6643 start
6856 start
4641 skipped
7186 start
6291 finish
7224 start
5803 skipped
7226 start

In [7]:
# 吐き出しておいたCSVを読み込む
df_own_stock = pd.read_csv("own_stock.csv")

In [ ]:
# データフレームに市場と業種を入れる
df_own_stock = append_market_and_industries(df_own_stock)

# スプレッドシートに書き込めるようにリスト形式に変換する
list_own_stock = df_own_stock.values.tolist()

In [ ]:
# 2つのAPIを記述しないとリフレッシュトークンを3600秒毎に発行し続けなければならない
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# 認証情報設定
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "keys/credentials.json", scope
)

In [ ]:
# OAuth2の資格情報を使用してGoogle APIにログイン
gc = gspread.authorize(credentials)

In [ ]:
# 共有設定したスプレッドシートを開く
spreadsheet = gc.open_by_key(config.SPREADSHEET_KEY)

In [ ]:
# シート名を指定してシートを開く
worksheet = spreadsheet.worksheet("②高配当株ポートフォリオ")

In [ ]:
# シートの既存の値をクリアする（B列からJ列だけ上書きするため）
worksheet.batch_clear(["B4:I300"])

In [ ]:
# シートのB4:J4からデータフレームの値を書き込む
cell_range = f"B4:I{3 + len(list_own_stock)}"
worksheet.update(cell_range, list_own_stock)